In [1]:
import numpy as np
import copy
from nibabel import trackvis as tv
from dipy.tracking.streamline import Streamlines
from dipy.segment.clustering import QuickBundles
from dipy.io.pickles import save_pickle
from dipy.data import get_fnames
import vtk
from dipy.viz import window, actor
from dipy.tracking.streamline import Streamlines
from dipy.io.streamline import load_trk, save_trk
from dipy.segment.metric import ResampleFeature, AveragePointwiseEuclideanMetric
from dipy.io.image import load_nifti
from dipy.tracking import utils
from dipy.viz import window, actor
from time import sleep
from dipy.data import two_cingulum_bundles
from dipy.align.streamlinear import StreamlineLinearRegistration
from dipy.tracking.streamline import set_number_of_points

In [2]:
def show_both_bundles(bundles, colors=None, show=True, fname=None):

    ren = window.Renderer()
    ren.SetBackground(1., 1, 1)
    for (i, bundle) in enumerate(bundles):
        color = colors[i]
        lines_actor = actor.streamtube(bundle, color, linewidth=0.05)
        #lines_actor.RotateX(-90)
        #lines_actor.RotateZ(90)
        ren.add(lines_actor)
    if show:
        window.show(ren)
    if fname is not None:
        sleep(1)
        window.record(ren, n_frames=1, out_path=fname, size=(900, 900))

In [3]:
#indicate the ROIs interested, note target_l>target_r
target_l = 51
target_r = 1051

In [4]:
#general information
l = ['N54717','N54718','N54719','N54720','N54722','N54759','N54760','N54761','N54762','N54763','N54764','N54765','N54766','N54770','N54771','N54772','N54798','N54801','N54802','N54803','N54804','N54805','N54806','N54807','N54818','N54824','N54825','N54826','N54837','N54838','N54843','N54844','N54856','N54857','N54858','N54859','N54860','N54861','N54873','N54874','N54875','N54876','N54877','N54879','N54880','N54891','N54892','N54893','N54897','N54898','N54899','N54900','N54915','N54916','N54917']
# gen4idx = [1,2,3,4,7,8,9,10,12,13,52,53,54]
# gen3idx = [14,15,16,17,18,19,30,21,22,23]
# gen4 = [l[i] for i in gen4idx]
# gen3 = [l[j] for j in gen3idx]
# gen = [gen3, gen4]

#exclude N54900
gen0idx = [5,6,11,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50]
gen0 = [l[k] for k in gen0idx]
# oldidx = [5,6,11,24,25,26,27,28,29,30,31,32,33,34,45,46,47,48,49,50]
# old = [l[o] for o in oldidx]
oldselectidx = [6, 25, 27, 28, 29, 32, 33, 45, 46, 50]
old = [l[o] for o in oldselectidx]
youngidx = [35,36,37,38,39,40,41,42,43,44]
young = [l[y] for y in youngidx]
age = [young,old]


In [5]:
# import random
# random.seed(123)
# oldidx = random.sample(oldidx,10)
# old = [l[o] for o in oldidx]
# age = [young,old]

In [6]:
#set path
mypath = '/Users/alex/code/Wenlin/data'
outpath = '/Users/alex/code/Wenlin/Tracts_Registration/results'

In [7]:
#set parameter
num_points1 = 100
distance1 = 1
feature1 = ResampleFeature(nb_points=num_points1)
metric1 = AveragePointwiseEuclideanMetric(feature=feature1)

#group cluster parameter
num_points2 = 100
distance2 = 1
feature2 = ResampleFeature(nb_points=num_points2)
metric2 = AveragePointwiseEuclideanMetric(feature=feature2) 

In [8]:
#load the control animal
streams_control,hdr_control = load_trk(mypath+'/wenlin_results/N54900_bmCSA_detr_small.trk')
labels_control, affine_labels_control = load_nifti(mypath+'/wenlin_data/labels/fa_labels_warp_N54900_RAS.nii.gz') 

In [9]:
#uncomment this cell to generate both left and right
# labels_pair_control = copy.copy(labels_control)
# nonz_control = np.nonzero(labels_pair_control)
# for i in range(len(nonz_control[0])):
#     if labels_pair_control[nonz_control[0][i], nonz_control[1][i], nonz_control[2][i]]>=1000:
#         labels_pair_control[nonz_control[0][i], nonz_control[1][i], nonz_control[2][i]] -= 1000
# print('pair labels generated')

# #pair labels target control animals
# streams_fix_control = lambda : (sl for sl in streams_control if len(sl)>1)
# streamlines_control = Streamlines(streams_fix_control())
# M_control, grouping_control = utils.connectivity_matrix(streamlines_control, labels_pair_control, 
#                                                         affine=affine_labels_control, return_mapping=True,
#                                                         mapping_as_streamlines=True)

# target_streamlines_control = grouping_control[target_l, target_r]

In [10]:
#target control animals
streams_fix_control = lambda : (sl for sl in streams_control if len(sl)>1)
streamlines_control = Streamlines(streams_fix_control())
M_control, grouping_control = utils.connectivity_matrix(streamlines_control, labels_control, 
                                                        affine=affine_labels_control, return_mapping=True,
                                                        mapping_as_streamlines=True)

target_streamlines_control = grouping_control[target_l, target_r]

In [11]:
#cluster control animals
target_qb_control = QuickBundles(threshold=distance1,metric=metric1)
target_clusters_control = target_qb_control.cluster(target_streamlines_control)
print("Control Nb. clusters:", len(target_clusters_control))

Control Nb. clusters: 117


In [12]:
#group calculation
for k in range(2):
    agegroup = age[k]
    createVar = locals()
    createVar['groupclusters'+str(k+1)] = [] #create empty list, 2-old, 1-young
    animallist = []
    print('Group'+str(k+1)+' started')
    for j in range(len(agegroup)):
        runno = agegroup[j]
        animallist.append(runno)
        streams,hdr = load_trk(mypath+'/wenlin_results/'+runno+'_bmCSA_detr_small.trk')
        labels, affine_labels = load_nifti(mypath+'/wenlin_data/labels/fa_labels_warp_'+runno+'_RAS.nii.gz') 

#         labels_pair = copy.copy(labels)
#         nonz = np.nonzero(labels_pair)
#         for i in range(len(nonz[0])):
#             if labels_pair[nonz[0][i], nonz[1][i], nonz[2][i]]>=1000:
#                 labels_pair[nonz[0][i], nonz[1][i], nonz[2][i]] -= 1000
#         #print('pair labels generated')
        
        #target moving animals
        streams_fix = lambda : (sl for sl in streams if len(sl)>1)
        streamlines = Streamlines(streams_fix())
        M, grouping = utils.connectivity_matrix(streamlines, labels, affine=affine_labels, 
                                                return_mapping=True,mapping_as_streamlines=True)

        target_streamlines = grouping[target_l, target_r]
        
        
        target_qb = QuickBundles(threshold=distance1,metric=metric1)
        target_clusters = target_qb.cluster(target_streamlines)
        print('NO.'+str(j+1)+' '+runno+" Nb. clusters:", len(target_clusters))
        #registration
        srr = StreamlineLinearRegistration()
        srm = srr.optimize(static=target_clusters_control.centroids, moving=target_clusters.centroids)
        target_subj_aligned = srm.transform(target_clusters.centroids)
        
        locals()['groupclusters'+str(k+1)].extend(target_subj_aligned)
    print('agetype-'+str(k+1)+' finished. '+'total number of clusters for group'+ str(k+1) 
          + ': {}'.format(len(locals()['groupclusters'+str(k+1)])))
    print('animal list: ', animallist)
    print(' ')
        
        

Group1 started
NO.1 N54859 Nb. clusters: 66


KeyboardInterrupt: 

In [56]:
group1_qb = QuickBundles(threshold=distance2,metric=metric2)
group1_clusters = group1_qb.cluster(groupclusters1)
group2_qb = QuickBundles(threshold=distance2,metric=metric2)
group2_clusters = group2_qb.cluster(groupclusters2)

In [57]:
from dipy.io.streamline import save_trk
save_trk('results/'+str(target_l)+'--'+str(target_r)+'11youngCSA.trk', group1_clusters.centroids, affine=np.eye(4),
         shape=labels.shape)
save_trk('results/'+str(target_l)+'--'+str(target_r)+'11oldCSA.trk', group2_clusters.centroids, affine=np.eye(4),
         shape=labels.shape)

In [13]:
target_subj_aligned

[array([[-3.10487052e+00, -3.67723380e+00, -5.32636338e-02],
        [-3.12042695e+00, -3.67775991e+00, -5.36502099e-02],
        [-3.13591085e+00, -3.67525963e+00, -5.49294617e-02],
        [-3.15739098e+00, -3.66930092e+00, -5.42117424e-02],
        [-3.18740600e+00, -3.64891801e+00, -5.22416783e-02],
        [-3.21825344e+00, -3.61367953e+00, -5.19481896e-02],
        [-3.24445874e+00, -3.56363691e+00, -5.36880130e-02],
        [-3.27093940e+00, -3.49885961e+00, -5.16698703e-02],
        [-3.29636553e+00, -3.42266127e+00, -4.31010681e-02],
        [-3.31583181e+00, -3.33716762e+00, -2.65109686e-02],
        [-3.32785311e+00, -3.24105899e+00, -6.20246152e-04],
        [-3.32897784e+00, -3.13832015e+00,  3.25512918e-02],
        [-3.32020481e+00, -3.03393730e+00,  7.08586956e-02],
        [-3.30477135e+00, -2.92906223e+00,  1.12838631e-01],
        [-3.28203062e+00, -2.82727551e+00,  1.61405113e-01],
        [-3.25380847e+00, -2.73303037e+00,  2.17123068e-01],
        [-3.21921820e+00

In [16]:
target_clusters_control.clusters[0]

Cluster([0, 1, 2, 4, 5, 6, 8, 15, 17, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 32, 33, 34, 35, 36, 38, 40, 41, 42, 43, 44, 46, 47, 48, 50, 51, 52, 58, 62, 63, 66, 67, 71, 72, 73, 74, 75, 79, 81, 82, 83, 84, 87, 89, 91, 93, 94, 95, 96, 97, 99, 101, 102, 103, 106, 109, 110, 111, 112, 113, 116, 117, 125, 135, 137, 139, 141, 143, 147, 149, 155, 157, 161, 164, 165, 166, 168, 169, 170, 176, 177, 178, 179, 180, 186, 187, 189, 190, 191, 193, 195, 196, 200, 201, 202, 203, 205, 206, 208, 209, 210, 215, 216, 223, 224, 226, 227, 232, 234, 235, 239, 240, 243, 244, 247, 256, 265, 267, 270, 273, 278, 279, 283, 284, 290, 293, 304, 308, 318, 333, 335, 340, 342, 347, 355, 363, 364, 371, 373, 383, 389, 391, 393, 395, 396, 401, 403, 405, 409, 434, 443, 447, 449, 450, 455, 456, 458, 459, 462, 472, 475, 483, 491, 496, 504, 511, 528, 535, 538, 549, 553, 556, 568, 574, 576, 584, 601, 629, 633, 636, 638, 683, 690, 718, 722, 726, 736, 744, 790, 792, 821, 857, 863, 868, 870, 879, 889, 912, 928, 947, 948, 970, 984